In [8]:
import cv2
import csv
import sklearn
import numpy as np

lines=[]
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(lines, test_size=0.2)

def generator(samples, batch_size=32):
    while 1:
        shuffle(samples)
        for batch in range(1, len(lines), batch_size):
            batch_samples = lines[batch:batch+batch_size]
            images=[]
            steering=[]
            for line in batch_samples:
                c_dir_path = line[0].split()
                image_path = 'data/' + str(c_dir_path[0])
                image=cv2.cvtColor(cv2.imread(image_path),cv2.COLOR_BGR2RGB)
                images.append(image)
                steer=float(line[3])
                steering.append(steer)
                images.append(cv2.flip(image,1))
                steering.append(steer*-1)
                images.append(cv2.add(cv2.multiply(image, np.array([1.5]))), 50.0)
                steering.append(steer)

                l_dir_path = line[1].split()
                image_path = 'data/' + str(l_dir_path[0])
                image=cv2.cvtColor(cv2.imread(image_path),cv2.COLOR_BGR2RGB)
                images.append(image)
                steer = steer + 0.2
                steering.append(steer)
                images.append(cv2.flip(image,1))
                steering.append(steer*-1)
                images.append(cv2.add(cv2.multiply(image, np.array([1.5]))), 50.0)
                steering.append(steer)

                r_dir_path = line[2].split()
                image_path = 'data/' + str(r_dir_path[0])
                image=cv2.cvtColor(cv2.imread(image_path),cv2.COLOR_BGR2RGB)
                images.append(image)
                steer = steer - 0.2
                steering.append(steer)
                images.append(cv2.flip(image,1))
                steering.append(steer*-1)
                images.append(cv2.add(cv2.multiply(image, np.array([1.5]))), 50.0)
                steering.append(steer)

            X_train = np.array(images)
            y_train = np.array(steering)
            print(len(X_train), len(y_train))
            yield shuffle(X_train, y_train)    

In [ ]:
Xtrain_generator = generator(train_sample)
ytrain_generator = generator(y_train)

In [ ]:
import keras.models import Sequential
import keras.layers import Flatten, Dense, Cropping2D

model = Sequential()
model.add(Cropping2D(cropping=((50,20), (0,0))), input_shape=(3,160,320))
model.add(Lambda(lambda x: x/255.0 - 0.5))
model.add(Flatten()))
model.add(Dense(1))
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_train, y_train, validation_split = 0.2, shuffle = true, nb_epoch=3)

model.save('model.h5')